# 6

In [1]:
rdd = sc.parallelize([1,2,3,4,5])

In [2]:
rdd

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:195

In [3]:
doubled = rdd.map(lambda x: x * 2)

In [4]:
doubled

PythonRDD[1] at RDD at PythonRDD.scala:53

In [6]:
doubled.toDebugString()

b'(4) PythonRDD[1] at RDD at PythonRDD.scala:53 []\n |  ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:195 []'

# 7

In [3]:
import pyspark
import os
#spark_location='/opt/spark-2.4.3/' # Set your own
java8_location= '/usr/lib/jvm/java-8-openjdk-amd64' # Set your own
os.environ['JAVA_HOME'] = java8_location
#findspark.init(spark_home=spark_location) 

In [1]:
import pyspark

In [5]:
from pyspark import SparkContext
sc =SparkContext()

In [6]:
text = sc.textFile('Working Files/03/Audio Standardization Sentences.txt')

In [7]:
text

Working Files/03/Audio Standardization Sentences.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [8]:
text.collect()

['Oak is strong and also gives shade.',
 'Cats and dogs each hate the other.',
 'The pipe began to rust while new.',
 "Open the crate but don't break the glass.",
 'Add the sum to the product of these three.',
 'Thieves who rob friends deserve jail.',
 'The ripe taste of cheese improves with age.',
 'Act on these orders with great speed.',
 'The hog crawled under the high fence.',
 'Move the vat over the hot fire.']

In [9]:
text = sc.textFile('Working Files/03/Audio Standardization Sentences.txt', minPartitions=5)
# set utf-8 encoding for faster processing

# 8

In [11]:
numbers = sc.parallelize(range(20)) # numbers rdd

In [12]:
#Pulling all the data back to the Driver
numbers.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [13]:
numbers.sum()

190

In [14]:
# returns first n elements
numbers.take(3)

[0, 1, 2]

In [15]:
numbers.count()

20

In [16]:
# we have 4 cores -> numbers will be split into 4 text files each containing 20 / 4 = 5 elements
numbers.saveAsTextFile('Working Files/03/test_numbers.txt')

Typical rule:

    1) Function takes RDD and returns RDD -> it is Transformation
    
    2) Function takes RDD and returns something ELSE -> it is Action

# 9

In [19]:
numbers = sc.parallelize(range(20)) # numbers rdd
numbers.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [23]:
numbers.toDebugString()

b'(4) PythonRDD[15] at collect at <ipython-input-19-715c5a507793>:2 []\n |  ParallelCollectionRDD[14] at parallelize at PythonRDD.scala:195 []'

In [20]:
small_numbers = sc.parallelize(range(5))
small_numbers.collect()

[0, 1, 2, 3, 4]

In [24]:
small_numbers.toDebugString()

b'(4) PythonRDD[17] at collect at <ipython-input-20-40395621a053>:2 []\n |  ParallelCollectionRDD[16] at parallelize at PythonRDD.scala:195 []'

In [25]:
combined = numbers.union(small_numbers)
combined.toDebugString()

b'(8) UnionRDD[19] at union at NativeMethodAccessorImpl.java:0 []\n |  PythonRDD[15] at collect at <ipython-input-19-715c5a507793>:2 []\n |  ParallelCollectionRDD[14] at parallelize at PythonRDD.scala:195 []\n |  PythonRDD[17] at collect at <ipython-input-20-40395621a053>:2 []\n |  ParallelCollectionRDD[16] at parallelize at PythonRDD.scala:195 []'

# 10

In [26]:
inputs = sc.parallelize(range(1000))

In [27]:
# some expensive calculation
result = inputs.map(lambda x: x ** 5)

In [28]:
result.saveAsTextFile('Working Files/03/10_first-output.txt')

In [31]:
# here the values obtained in "some expensive calculation" might be lost, because we've
# already called an action in the cell above -> persistance
result.map(lambda x: "number:" + str(x)).saveAsTextFile('Working Files/03/10_second-output.txt')

Serializing objects can save a lot of memory

Replication level -> more nodes can use this chunk of data -> less chances we loose it in case we lose a node 

In [33]:
# we will not need to re-calculate every time we need it
inputs.persist(pyspark.StorageLevel.MEMORY_ONLY)
# If we use RDD more than once consider doing persistence
# https://spark.apache.org/docs/1.5.0/api/python/_modules/pyspark/storagelevel.html

PythonRDD[33] at RDD at PythonRDD.scala:53

In [34]:
inputs.is_cached # check whether it is persisted

True

In [36]:
inputs.unpersist()

PythonRDD[33] at RDD at PythonRDD.scala:53

In [37]:
inputs.is_cached

False